### Visualize W&B results

In [ ]:
import pandas as pd
import wandb
wandb.login()

wandb: Currently logged in as: simonkral99 (uni-ma) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [2]:
api = wandb.Api()

# Replace with your actual W&B username and project name
runs = api.runs("simonkral99/Thesis_MTSF")

In [ ]:
for run in runs:
    print(run.name)
    print(run.config)   # hyperparameters
    print(run.summary)  # final logged metrics
    print(run.config.keys())

run_DLinear_custom_104
{'des': 'Exp', 'gpu': 0, 'itr': 1, 'd_ff': 2048, 'data': 'custom', 'freq': 'h', 'loss': 'mse', 'c_out': 7, 'embed': 'timeF', 'lradj': 'type3', 'model': 'DLinear', 'revin': 1, 'affine': 0, 'dec_in': 7, 'distil': True, 'enc_in': 7, 'factor': 1, 'stride': 8, 'target': 'OT', 'd_model': 512, 'devices': '0,1,2,3', 'dropout': 0.05, 'n_heads': 8, 'seq_len': 104, 'use_amp': False, 'use_gpu': True, 'd_layers': 1, 'e_layers': 2, 'features': 'M', 'model_id': 'national_illness_104_24', 'patience': 100, 'pred_len': 24, 'data_path': 'national_illness.csv', 'label_len': 18, 'patch_len': 16, 'pct_start': 0.3, 'root_path': './dataset/', 'test_flop': False, 'activation': 'gelu', 'batch_size': 32, 'do_predict': True, 'embed_type': 0, 'fc_dropout': 0.05, 'individual': 0, 'moving_avg': 25, 'checkpoints': './checkpoints/', 'is_training': 1, 'kernel_size': 25, 'num_workers': 10, 'random_seed': 2021, 'head_dropout': 0, 'train_epochs': 100, 'decomposition': 0, 'learning_rate': 0.01, 'padd

In [10]:

run_data = []

for run in runs:
    summary = run.summary._json_dict
    config = {k: v for k, v in run.config.items() if not k.startswith('_')}
    run_data.append({**config, **summary, "run_id": run.id, "name": run.name})

df = pd.DataFrame(run_data)


columns_to_keep = ['model', 'pred_len', 'random_seed', 'data_path', 'mae', 'mse']

# Keep only those columns
df_subset = df[columns_to_keep]
df_subset['data_path'] = df_subset['data_path'].str.replace('.csv', '', regex=False)

display(df_subset)
#print(df.columns)

/scratch/ipykernel_1075057/4268229683.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['data_path'] = df_subset['data_path'].str.replace('.csv', '', regex=False)


,model,pred_len,random_seed,data_path,mae,mse
0,DLinear,24,2021,national_illness,0.974513,1.963897
1,DLinear,96,2021,ETTh1,0.395379,0.372835
2,DLinear,96,2021,ETTh2,0.344102,0.278666
3,DLinear,96,2021,ETTm1,0.346427,0.302771
4,DLinear,96,2021,traffic,NaN,NaN
5,DLinear,36,2021,national_illness,0.998045,2.080387
6,DLinear,48,2021,national_illness,1.002502,2.064065
7,DLinear,60,2021,national_illness,1.075969,2.273535
8,DLinear,192,2021,ETTm1,NaN,NaN
